In [3]:
import soundfile as sf
from pathlib import Path

In [5]:
WAVDIR = "/home/joregan/ljspeech/LJSpeech-1.1/wav16"
OUTDIR = "/home/joregan/ljspeech/data/"

In [9]:
WAVPATH = Path(WAVDIR)
OUTPATH = Path(OUTDIR)
IDSPATH = Path("/home/joregan/ljspeech/ids/")

In [6]:
frames = {}
for wf in WAVPATH.glob("*.wav"):
    stem = wf.stem
    data, sr = sf.read(str(wf))
    frames[stem] = len(data)

In [15]:
train_frames = {}
train_file = OUTPATH / "train.tsv"
with open(str(train_file)) as f:
    for line in f.readlines():
        if not "\t" in line:
            continue
        pieces = line.strip().split("\t")
        assert len(pieces) == 2
        train_frames[pieces[0]] = int(pieces[1])

In [22]:
MINS = [i * 5 for i in range(1, 25)]
HOURS = [i * 2 for i in range(1, 9)]

In [25]:
for min in MINS:
    frames = min * 60 * 16000
    idlist = [k for k in train_frames.keys()]
    outtsv = OUTPATH / f"{min}mins.tsv"
    with open(outtsv, "w") as of:
        current = 0
        of.write(f"{WAVDIR}\n")
        while frames > 0 and frames > current:
            id = idlist.pop(0)
            current = train_frames[id]
            of.write(f"{id}\t{current}\n")
            frames = frames - current
        max = 0
        maxid = ""
        for id in idlist:
            time = train_frames[id]
            if time > current:
                continue
            if time > max:
                max = time
                maxid = id
        of.write(f"{maxid}\t{max}\n")

In [ ]:
for min in HOURS:
    frames = min * 60 * 60 * 16000
    idlist = [k for k in train_frames.keys()]
    outtsv = OUTPATH / f"{min}hrs.tsv"
    with open(outtsv, "w") as of:
        current = 0
        of.write(f"{WAVDIR}\n")
        while frames > 0 and frames > current:
            id = idlist.pop(0)
            current = train_frames[id]
            of.write(f"{id}\t{current}\n")
            frames = frames - current
        max = 0
        maxid = ""
        for id in idlist:
            time = train_frames[id]
            if time > current:
                continue
            if time > max:
                max = time
                maxid = id
        of.write(f"{maxid}\t{max}\n")

In [12]:
for idfile in IDSPATH.glob("*.ids"):
    stem = idfile.stem
    outtsv = OUTPATH / f"{stem}.tsv"
    print("Current: ", stem)
    with open(str(idfile)) as f, open(outtsv, "w") as of:
        of.write(f"{WAVDIR}\n")
        for line in f.readlines():
            curstem = line.strip()
            of.write(f"{curstem}\t{train_frames[curstem]}\n")
            

Current:  20min
Current:  4h


KeyError: '121600'